In [ ]:
import cupy as cp
import pandas as pd
from cupyx.scipy.linalg import pinv as cp_pinv
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from tqdm import tqdm

# Data setup
n_samples = 445
n_features = 1
lambda_values = cp.logspace(-3, 0, 50)  # Move to GPU
data = pd.read_csv('LaLonde.csv')
X = data['treat'].values.reshape(n_samples, n_features)
Y = data['re78'].values
W = data['re75'].values.reshape(n_samples, n_features)

# Standardize X and W (on CPU, then transfer to GPU)
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_W = StandardScaler()
W_scaled = scaler_W.fit_transform(W)

# Transfer data to GPU
X_gpu = cp.array(X_scaled)
W_gpu = cp.array(W_scaled)
Y_gpu = cp.array(Y)

# Compute bandwidth parameters (on CPU due to pdist)
distances_X = pdist(X_scaled, metric='euclidean')
sigma_X = cp.median(cp.array(distances_X))  # Move result to GPU
distances_W = pdist(W_scaled, metric='euclidean')
sigma_W = cp.median(cp.array(distances_W))  # Move result to GPU
print(f"sigma_X: {float(sigma_X)}, sigma_W: {float(sigma_W)}")

# Set up kernel functions (manual implementation for GPU)
def compute_gram_matrices(X, W, sigma_X, sigma_W):
    # Compute pairwise squared distances on GPU
    X_diff = X[:, None] - X[None, :]
    W_diff = W[:, None] - W[None, :]
    K_X = cp.exp(-cp.sum(X_diff**2, axis=2) / (2 * sigma_X**2))
    K_W = cp.exp(-cp.sum(W_diff**2, axis=2) / (2 * sigma_W**2))
    return K_X, K_W

# Feature function phi(Y, t)
def phi(Y, t):
    return cp.exp(1j * t * Y)

# Weight function m(X, s)
def m(X, s):
    return cp.exp(1j * s * X)

# Estimate H(W, t)
def estimate_H(t, lambda_val, K_X, K_W, Y):
    phi_Y_t = phi(Y, t)
    matrix = K_W @ K_X @ K_W + lambda_val * n_samples**2 * K_W
    alpha = cp_pinv(matrix) @ (K_W @ K_X @ phi_Y_t)
    return alpha

# Cross-validation for lambda (simplified for GPU)
def cross_validate_lambda(t, X, Y, W, K_X, K_W):
    kf = KFold(n_splits=5)
    best_lambda = None
    best_score = cp.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X.get()):  # .get() to use CPU indices
            X_train = X[train_idx]
            X_val = X[val_idx]
            Y_train = Y[train_idx]
            Y_val = Y[val_idx]
            W_train = W[train_idx]
            W_val = W[val_idx]
            K_X_train = K_X[train_idx][:, train_idx]
            K_W_train = K_W[train_idx][:, train_idx]
            K_W_val_train = K_W[val_idx][:, train_idx]
            alpha = estimate_H(t, lambda_val, K_X_train, K_W_train, Y_train)
            H_W_val = K_W_val_train @ alpha
            Delta_val = phi(Y_val, t) - H_W_val
            score = cp.mean(cp.abs(Delta_val)**2)
            scores.append(score)
        avg_score = cp.mean(cp.array(scores))
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda

# Compute residuals U
def compute_U(t, alpha, K_W, Y):
    H_W_t = K_W @ alpha
    U = phi(Y, t) - H_W_t
    return U

# Compute T_n(s, t) vectorized
def compute_Tn(s_values, t, U, X):
    m_X_s = m(X, s_values[:, None])  # Broadcast s_values
    Tn = (1 / cp.sqrt(n_samples)) * cp.sum(U * m_X_s, axis=1)
    return Tn

# Trapezoidal integration (since simpson is not in CuPy)
def trapz(y, x):
    dx = x[1:] - x[:-1]
    return cp.sum((y[:-1] + y[1:]) * dx / 2)

# Compute Delta_phi_m
def compute_Delta_phi_m(X, Y, W, t_values, s_values):
    K_X, K_W = compute_gram_matrices(X, W, sigma_X, sigma_W)
    Delta_values = []
    for t in tqdm(t_values, desc="Computing Delta for t_values"):
        best_lambda = cross_validate_lambda(t, X, Y, W, K_X, K_W)
        alpha = estimate_H(t, best_lambda, K_X, K_W, Y)
        U = compute_U(t, alpha, K_W, Y)
        Tn_values = compute_Tn(s_values, t, U, X)
        Tn_abs_squared = cp.abs(Tn_values)**2
        integral_approx = trapz(Tn_abs_squared, s_values)
        Delta_values.append(integral_approx)
    return cp.max(cp.array(Delta_values))

# Parallel bootstrap (runs on CPU, calls GPU functions)
def parallel_bootstrap(idx, X, Y, W, t_values, s_values):
    X_boot = X[idx]
    Y_boot = Y[idx]
    W_boot = W[idx]
    return compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values).get()

# Bootstrap p-value estimation
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps):
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values)
    indices_list = [cp.random.choice(n_samples, n_samples, replace=True).get() for _ in range(n_bootstraps)]
    
    with Parallel(n_jobs=-1) as parallel:
        bootstrap_Deltas = parallel(
            delayed(parallel_bootstrap)(idx, X, Y, W, t_values, s_values)
            for idx in indices_list
        )
    
    bootstrap_Deltas = cp.array(bootstrap_Deltas)
    p_value = cp.mean(bootstrap_Deltas >= Delta_obs).get()
    return p_value, Delta_obs.get(), bootstrap_Deltas.get()

# Setup t_values and s_values
std_Y = cp.std(Y_gpu).get()
std_X = cp.std(X_gpu).get()
t_values = cp.linspace(-2*std_Y, 2*std_Y, 100)
s_values = cp.linspace(-2*std_X, 2*std_X, 100)

# Run experiments
n_experiments = 10
p_values = []
for i in tqdm(range(n_experiments), desc="Running experiments"):
    cp.random.seed(i)
    p_value, Delta_obs, bootstrap_Deltas = bootstrap_p_value(X_gpu, Y_gpu, W_gpu, t_values, s_values, n_bootstraps=100)
    p_values.append(p_value)
    tqdm.write(f"Experiment {i+1}: p-value = {p_value}")

avg_p_value = cp.mean(cp.array(p_values)).get()
rejection_rate = cp.mean(cp.array([1 if p < 0.05 else 0 for p in p_values])).get()
print(f"Average p-value: {avg_p_value}, Rejection rate: {rejection_rate}")

# Visualize (transfer to CPU for plotting)
plt.hist(bootstrap_Deltas, bins=30, density=True, alpha=0.7, label='Bootstrap Delta')
plt.axvline(Delta_obs, color='r', linestyle='--', label=f'Observed Delta = {Delta_obs:.2f}')
plt.title('Bootstrap Distribution of Delta')
plt.xlabel('Delta')
plt.ylabel('Density')
plt.legend()
plt.show()

In [18]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from tqdm import tqdm

# Data setup
n_samples = 445
n_features = 1
#lambda_values = np.array([0.1])
lambda_values = np.logspace(-3, 0, 50)
# Load LaLonde dataset
data = pd.read_csv('LaLonde.csv')
X = data['treat'].values.reshape(n_samples, n_features)
Y = data['re78'].values
W = data['re75'].values.reshape(n_samples, n_features)

# Standardize X and W
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_W = StandardScaler()
W_scaled = scaler_W.fit_transform(W)

# Compute bandwidth parameters
distances_X = pdist(X_scaled, metric='euclidean')
sigma_X = np.median(distances_X)
distances_W = pdist(W_scaled, metric='euclidean')
sigma_W = np.median(distances_W)
print(f"sigma_X: {sigma_X}, sigma_W: {sigma_W}")

# Set up kernel functions
kernel_X = RBF(length_scale=1.0)
kernel_W = RBF(length_scale=sigma_W)

# Feature function phi(Y, t)
def phi(Y, t):
    return np.exp(1j * t * Y)

# Weight function m(X, s)
def m(X, s):
    return np.exp(1j * s * X)

# Compute Gram matrices
def compute_gram_matrices(X, W):
    K_X = kernel_X(X)
    K_W = kernel_W(W)
    return K_X, K_W

# Estimate H(W, t)
def estimate_H(t, lambda_val, K_X, K_W, Y):
    phi_Y_t = phi(Y, t)
    matrix = K_W @ K_X @ K_W + lambda_val * n_samples**2 * K_W
    alpha = pinv(matrix) @ (K_W @ K_X @ phi_Y_t)
    return alpha

# Cross-validation for lambda
def cross_validate_lambda(t, X, Y, W):
    kf = KFold(n_splits=5)
    best_lambda = None
    best_score = np.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            W_train, W_val = W[train_idx], W[val_idx]
            K_X_train = kernel_X(X_train)
            K_W_train = kernel_W(W_train)
            K_W_val_train = kernel_W(W_val, W_train)
            alpha = estimate_H(t, lambda_val, K_X_train, K_W_train, Y_train)
            H_W_val = K_W_val_train @ alpha
            Delta_val = phi(Y_val, t) - H_W_val
            score = np.mean(np.abs(Delta_val)**2)
            scores.append(score)
        avg_score = np.mean(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda

# Compute residuals U
def compute_U(t, alpha, K_W, Y):
    H_W_t = K_W @ alpha
    U = phi(Y, t) - H_W_t
    return U

# Compute T_n(s, t)
def compute_Tn(s, t, U, X):
    m_X_s = m(X, s)
    Tn = (1 / np.sqrt(n_samples)) * np.sum(U * m_X_s)
    return Tn

# Compute Delta_phi_m with progress bar
def compute_Delta_phi_m(X, Y, W, t_values, s_values):
    K_X, K_W = compute_gram_matrices(X, W)
    Delta_values = []
    for t in tqdm(t_values, desc="Computing Delta for t_values"):
        best_lambda = cross_validate_lambda(t, X, Y, W)
        alpha = estimate_H(t, best_lambda, K_X, K_W, Y)
        U = compute_U(t, alpha, K_W, Y)
        Tn_values = [compute_Tn(s, t, U, X) for s in s_values]
        Tn_abs_squared = np.abs(Tn_values)**2
        integral_approx = simpson(Tn_abs_squared, s_values)
        Delta_values.append(integral_approx)
    return max(Delta_values)

# Parallel bootstrap computation
def parallel_bootstrap(idx, X, Y, W, t_values, s_values):
    X_boot = X[idx]
    Y_boot = Y[idx]
    W_boot = W[idx]
    return compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values)

# Bootstrap p-value estimation with parallel processing
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps):
    #np.random.seed(42)
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values)
    indices_list = [np.random.choice(n_samples, n_samples, replace=True) for _ in range(n_bootstraps)]
    
    # Use all logical cores for parallel computation
    with Parallel(n_jobs=-1) as parallel:
        bootstrap_Deltas = parallel(
            delayed(parallel_bootstrap)(idx, X, Y, W, t_values, s_values)
            for idx in indices_list
        )
    
    p_value = np.mean(np.array(bootstrap_Deltas) >= Delta_obs)
    return p_value, Delta_obs, bootstrap_Deltas

    

sigma_X: 0.0, sigma_W: 0.17753388125109104


In [ ]:
std_Y = np.std(Y)
std_X = np.std(X)
t_values = np.linspace(-2*std_Y, 2*std_Y, 100)
s_values = np.linspace(-2*std_X, 2*std_X, 100)

n_experiments = 10
p_values = []
for i in tqdm(range(n_experiments), desc="Running experiments"):
    np.random.seed(i)
    p_value, Delta_obs, bootstrap_Deltas = bootstrap_p_value(X_scaled, Y, W_scaled, t_values, s_values, n_bootstraps=100)
    p_values.append(p_value)
    tqdm.write(f"Experiment {i+1}: p-value = {p_value}")

avg_p_value = np.mean(p_values)
rejection_rate = np.mean([1 if p < 0.05 else 0 for p in p_values])
print(f"Average p-value: {avg_p_value}, Rejection rate: {rejection_rate}")

Computing Delta for t_values: 100%|██████████| 100/100 [13:17<00:00,  7.98s/it]


In [ ]:
# Visualize Bootstrap distribution
plt.hist(bootstrap_Deltas, bins=30, density=True, alpha=0.7, label='Bootstrap Delta')
plt.axvline(Delta_obs, color='r', linestyle='--', label=f'Observed Delta = {Delta_obs:.2f}')
plt.title('Bootstrap Distribution of Delta')
plt.xlabel('Delta')
plt.ylabel('Density')
plt.legend()
plt.show()

In [8]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from tqdm import tqdm

# Data setup
n_samples = 445
n_features = 1
#lambda_values = np.logspace(-3, 1, 10)
lambda_values=np.array([0.01,0.1])

# Load LaLonde dataset
data = pd.read_csv('LaLonde.csv')
X = data['treat'].values.reshape(n_samples, n_features)
Y = data['re78'].values
W = data['re75'].values.reshape(n_samples, n_features)

# Standardize X and W
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_W = StandardScaler()
W_scaled = scaler_W.fit_transform(W)

# Compute bandwidth parameters
distances_X = pdist(X_scaled, metric='euclidean')
sigma_X = np.median(distances_X)
distances_W = pdist(W_scaled, metric='euclidean')
sigma_W = np.median(distances_W)
print(f"sigma_X: {sigma_X}, sigma_W: {sigma_W}")

# Set up kernel functions
kernel_X = RBF(length_scale=1.0)
kernel_W = RBF(length_scale=sigma_W)

# Feature function phi(Y, t)
def phi(Y, t):
    return np.exp(1j * t * Y)

# Weight function m(X, s)
def m(X, s):
    return np.exp(1j * s * X)

# Compute Gram matrices
def compute_gram_matrices(X, W):
    K_X = kernel_X(X)
    K_W = kernel_W(W)
    return K_X, K_W

# Estimate H(W, t)
def estimate_H(t, lambda_val, K_X, K_W, Y):
    phi_Y_t = phi(Y, t)
    matrix = K_W @ K_X @ K_W + lambda_val * n_samples**2 * K_W
    alpha = pinv(matrix) @ (K_W @ K_X @ phi_Y_t)
    return alpha

# Cross-validation for lambda
def cross_validate_lambda(t, X, Y, W):
    kf = KFold(n_splits=5)
    best_lambda = None
    best_score = np.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            W_train, W_val = W[train_idx], W[val_idx]
            K_X_train = kernel_X(X_train)
            K_W_train = kernel_W(W_train)
            K_W_val_train = kernel_W(W_val, W_train)
            alpha = estimate_H(t, lambda_val, K_X_train, K_W_train, Y_train)
            H_W_val = K_W_val_train @ alpha
            Delta_val = phi(Y_val, t) - H_W_val
            score = np.mean(np.abs(Delta_val)**2)
            scores.append(score)
        avg_score = np.mean(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda

# Compute residuals U
def compute_U(t, alpha, K_W, Y):
    H_W_t = K_W @ alpha
    U = phi(Y, t) - H_W_t
    #print("U dtype:", U.dtype)  # 检查数据类型
    return U

# Compute T_n(s, t)
def compute_Tn(s, t, U, X):
    m_X_s = m(X, s)
    Tn = (1 / np.sqrt(n_samples)) * np.sum(U * m_X_s)
    #print("Tn dtype:", Tn.dtype)  # 检查数据类型
    return Tn

# Compute Delta_phi_m with progress bar
def compute_Delta_phi_m(X, Y, W, t_values, s_values):
    K_X, K_W = compute_gram_matrices(X, W)
    Delta_values = []
    for t in tqdm(t_values, desc="Computing Delta for t_values"):
        best_lambda = cross_validate_lambda(t, X, Y, W)
        alpha = estimate_H(t, best_lambda, K_X, K_W, Y)
        U = compute_U(t, alpha, K_W, Y)
        Tn_values = [compute_Tn(s, t, U, X) for s in s_values]
        Tn_abs_squared = np.abs(Tn_values)**2
        integral_approx = simpson(Tn_abs_squared, s_values)
        Delta_values.append(integral_approx)
    return max(Delta_values)

# Bootstrap p-value estimation with progress bar
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps=100):
    np.random.seed(42)
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values)
    indices_list = [np.random.choice(n_samples, n_samples, replace=True) for _ in range(n_bootstraps)]
    bootstrap_Deltas = []
    for idx in tqdm(indices_list, desc="Bootstrap iterations"):
        X_boot = X[idx]
        Y_boot = Y[idx]
        W_boot = W[idx]
        Delta_boot = compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values)
        bootstrap_Deltas.append(Delta_boot)
    p_value = np.mean(np.array(bootstrap_Deltas) >= Delta_obs)
    return p_value, Delta_obs, bootstrap_Deltas

# Main execution with progress bar
if __name__ == "__main__":
    std_Y = np.std(Y)
    std_X = np.std(X)
    t_values = np.linspace(-2*std_Y, 2*std_Y, 100)
    s_values = np.linspace(-2*std_X, 2*std_X, 100)

    n_experiments = 10
    p_values = []
    for i in tqdm(range(n_experiments), desc="Running experiments"):
        np.random.seed(i)
        p_value, Delta_obs, bootstrap_Deltas = bootstrap_p_value(X_scaled, Y, W_scaled, t_values, s_values, n_bootstraps=100)
        p_values.append(p_value)
        #print(f"Experiment {i+1}: p-value = {p_value}")
        tqdm.write(f"Experiment {i+1}: p-value = {p_value}")  # 使用 tqdm.write 替代 print

    avg_p_value = np.mean(p_values)
    rejection_rate = np.mean([1 if p < 0.05 else 0 for p in p_values])
    print(f"Average p-value: {avg_p_value}, Rejection rate: {rejection_rate}")

    # Visualize Bootstrap distribution
    plt.hist(bootstrap_Deltas, bins=30, density=True, alpha=0.7, label='Bootstrap Delta')
    plt.axvline(Delta_obs, color='r', linestyle='--', label=f'Observed Delta = {Delta_obs:.2f}')
    plt.title('Bootstrap Distribution of Delta')
    plt.xlabel('Delta')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

sigma_X: 0.0, sigma_W: 0.17753388125109104


Computing Delta for t_values: 100%|██████████| 100/100 [00:49<00:00,  2.02it/s]





































































































Computing Delta for t_values: 100%|██████████| 100/100 [00:50<00:00,  2.00it/s]





































































































Computing Delta for t_values: 100%|██████████| 100/100 [00:48<00:00,  2.06it/s]





































































































Computing Delta for t_values: 100%|██████████| 100/100 [00:47<00:00,  2.09it/s]





































































































Computing Delta for t_values: 100%|██████████| 100/100 [00:53<00:00,  1.88it/s]






































































Running experiments:   0%|          | 0/10 [05:26<?, ?it/s]


KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd
from scipy.linalg import pinv  # 直接使用 pinv
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed

# 数据设置
n_samples = 445
n_features = 1
lambda_values = np.array([0.01, 0.1, 1, 10])  # 正则化参数范围


# 计算Gram矩阵
def compute_gram_matrices(X, W):
    K_X = kernel_X(X)
    K_W = kernel_W(W)
    return K_X, K_W


# 特征函数 phi(Y, t)
def phi(Y, t):
    return np.exp(1j * t * Y)


# 权重函数 m(X, s)
def m(X, s):
    return np.exp(1j * s * X)


# 使用伪逆估计 H(W, t)
def estimate_H(t, lambda_val, K_X, K_W, Y):
    phi_Y_t = phi(Y, t)
    matrix = K_W @ K_X @ K_W + lambda_val * n_samples ** 2 * K_W
    alpha = pinv(matrix) @ (K_W @ K_X @ phi_Y_t)
    return alpha


# 交叉验证选择最佳 lambda
def cross_validate_lambda(t, X, Y, W):
    kf = KFold(n_splits=5)
    best_lambda = None
    best_score = np.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            W_train, W_val = W[train_idx], W[val_idx]
            K_X_train = kernel_X(X_train)
            K_W_train = kernel_W(W_train)
            K_W_val_train = kernel_W(W_val, W_train)
            alpha = estimate_H(t, lambda_val, K_X_train, K_W_train, Y_train)
            H_W_val = K_W_val_train @ alpha
            Delta_val = phi(Y_val, t) - H_W_val
            score = np.mean(np.abs(Delta_val) ** 2)
            scores.append(score)
        avg_score = np.mean(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda


# 计算残差 U
def compute_U(t, alpha, K_W, Y):
    H_W_t = K_W @ alpha
    U = phi(Y, t) - H_W_t
    print("U dtype:", U.dtype)  # 检查数据类型
    print("U[:5]:", U[:5])      # 检查前5个值
    return U


# 计算 T_n(s, t)
def compute_Tn(s, t, U, X):
    m_X_s = m(X, s)
    Tn = (1 / np.sqrt(n_samples)) * np.sum(U * m_X_s)
    print("Tn dtype:", Tn.dtype)  # 检查数据类型
    print("Tn:", Tn)
    return Tn


# 计算 Delta_phi_m
def compute_Delta_phi_m(X, Y, W, t_values, s_values):
    K_X, K_W = compute_gram_matrices(X, W)
    Delta_values = []
    for t in t_values:
        best_lambda = cross_validate_lambda(t, X, Y, W)
        alpha = estimate_H(t, best_lambda, K_X, K_W, Y)
        U = compute_U(t, alpha, K_W, Y)
        integral_approx = 0
        for s in s_values:
            Tn = compute_Tn(s, t, U, X)
            integral_approx += np.abs(Tn) ** 2 * (s_values[1] - s_values[0])
        Delta_values.append(integral_approx)
    return max(Delta_values)


# 修改 bootstrap_p_value 为支持并行的函数
def parallel_bootstrap(indices, X, Y, W, t_values, s_values):
    X_boot = X[indices]
    Y_boot = Y[indices]
    W_boot = W[indices]
    return compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values)


# Bootstrap p值估计
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps=100):
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values)
    indices_list = [np.random.choice(n_samples, n_samples, replace=True) for _ in range(n_bootstraps)]
    # 使用逻辑核心数进行并行计算
    with Parallel(n_jobs=18) as parallel:
        bootstrap_Deltas = parallel(
            delayed(parallel_bootstrap)(idx, X, Y, W, t_values, s_values)
            for idx in indices_list
        )
    p_value = np.mean([1 if d >= Delta_obs else 0 for d in bootstrap_Deltas])
    return p_value


if __name__ == "__main__":
    # 加载 LaLonde 数据集
    data = pd.read_csv('LaLonde.csv')
    X = data['treat'].values.reshape(n_samples, n_features).astype(np.float32)
    Y = data['re78'].values.astype(np.float32)
    W = data['re75'].values.reshape(n_samples, n_features).astype(np.float32)

    # 提前标准化和计算带宽（仅一次计算）
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    scaler_W = StandardScaler()
    W_scaled = scaler_W.fit_transform(W)
    sigma_X = np.median(pdist(X_scaled, metric='euclidean'))
    sigma_W = np.median(pdist(W_scaled, metric='euclidean'))
    kernel_X = RBF(length_scale=1.0)
    kernel_W = RBF(length_scale=sigma_W)

    std_Y = np.std(Y)
    std_X = np.std(X)
    t_values = np.linspace(-2 * std_Y, 2 * std_Y, 100, dtype=np.float32)
    s_values = np.linspace(-2 * std_X, 2 * std_X, 100, dtype=np.float32)

    p_value = bootstrap_p_value(X, Y, W, t_values, s_values)
    print(f"p 值: {p_value}")
    if p_value < 0.05:
        print("拒绝零假设")
    else:
        print("未能拒绝零假设")
    

U dtype: complex128
U[:5]: [-1.26680999+0.37641872j -1.34547477-0.04821987j -1.01001685+0.73402355j
 -0.2959119 +0.9849861j   0.56719203+0.39362002j]
Tn dtype: complex128
Tn: (474.5468129413377-194.91851044417294j)
Tn dtype: complex128
Tn: (478.48771831349666-192.17642991877358j)
Tn dtype: complex128
Tn: (482.37323916201666-189.35643133704724j)
Tn dtype: complex128
Tn: (486.2018374051932-186.45961235602292j)
Tn dtype: complex128
Tn: (489.97200480799785-183.48715648158964j)
Tn dtype: complex128
Tn: (493.68221821206345-180.4402042952067j)
Tn dtype: complex128
Tn: (497.3310417843514-177.3199814889598j)
Tn dtype: complex128
Tn: (500.9170111370556-174.12772849371393j)
Tn dtype: complex128
Tn: (504.4386908062544-170.86469974088095j)
Tn dtype: complex128
Tn: (507.8947177300152-167.53219184806815j)
Tn dtype: complex128
Tn: (511.2837148458603-164.13153035676706j)
Tn dtype: complex128
Tn: (514.6043054604283-160.66406954779566j)
Tn dtype: complex128
Tn: (517.8551990983357-157.13116259462066j)
Tn 

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import pinv2 as pinv  # 更高效的伪逆
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed

# 数据设置
n_samples = 445
n_features = 1
lambda_values = np.array([0.01, 0.1, 1, 10])  # 正则化参数范围


# 计算Gram矩阵
def compute_gram_matrices(X, W):
    K_X = kernel_X(X)
    K_W = kernel_W(W)
    return K_X, K_W


# 特征函数 phi(Y, t)
def phi(Y, t):
    return np.exp(1j * t * Y)


# 权重函数 m(X, s)
def m(X, s):
    return np.exp(1j * s * X)


# 使用伪逆估计 H(W, t)
def estimate_H(t, lambda_val, K_X, K_W, Y):
    phi_Y_t = phi(Y, t)
    matrix = K_W @ K_X @ K_W + lambda_val * n_samples ** 2 * K_W
    alpha = pinv(matrix) @ (K_W @ K_X @ phi_Y_t)
    return alpha


# 交叉验证选择最佳 lambda
def cross_validate_lambda(t, X, Y, W):
    kf = KFold(n_splits=5)
    best_lambda = None
    best_score = np.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            W_train, W_val = W[train_idx], W[val_idx]
            K_X_train = kernel_X(X_train)
            K_W_train = kernel_W(W_train)
            K_W_val_train = kernel_W(W_val, W_train)
            alpha = estimate_H(t, lambda_val, K_X_train, K_W_train, Y_train)
            H_W_val = K_W_val_train @ alpha
            Delta_val = phi(Y_val, t) - H_W_val
            score = np.mean(np.abs(Delta_val) ** 2)
            scores.append(score)
        avg_score = np.mean(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda


# 计算残差 U
def compute_U(t, alpha, K_W, Y):
    H_W_t = K_W @ alpha
    U = phi(Y, t) - H_W_t
    return U


# 计算 T_n(s, t)
def compute_Tn(s, t, U, X):
    m_X_s = m(X, s)
    Tn = (1 / np.sqrt(n_samples)) * np.sum(U * m_X_s)
    return Tn


# 计算 Delta_phi_m
def compute_Delta_phi_m(X, Y, W, t_values, s_values):
    K_X, K_W = compute_gram_matrices(X, W)
    Delta_values = []
    for t in t_values:
        best_lambda = cross_validate_lambda(t, X, Y, W)
        alpha = estimate_H(t, best_lambda, K_X, K_W, Y)
        U = compute_U(t, alpha, K_W, Y)
        integral_approx = 0
        for s in s_values:
            Tn = compute_Tn(s, t, U, X)
            integral_approx += np.abs(Tn) ** 2 * (s_values[1] - s_values[0])
        Delta_values.append(integral_approx)
    return max(Delta_values)


# 修改 bootstrap_p_value 为支持并行的函数
def parallel_bootstrap(indices, X, Y, W, t_values, s_values):
    X_boot = X[indices]
    Y_boot = Y[indices]
    W_boot = W[indices]
    return compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values)


# Bootstrap p值估计
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps=100):
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values)
    indices_list = [np.random.choice(n_samples, n_samples, replace=True) for _ in range(n_bootstraps)]
    # 使用逻辑核心数进行并行计算
    with Parallel(n_jobs=18) as parallel:
        bootstrap_Deltas = parallel(
            delayed(parallel_bootstrap)(idx, X, Y, W, t_values, s_values)
            for idx in indices_list
        )
    p_value = np.mean([1 if d >= Delta_obs else 0 for d in bootstrap_Deltas])
    return p_value


if __name__ == "__main__":
    # 加载 LaLonde 数据集
    data = pd.read_csv('LaLonde.csv')
    X = data['treat'].values.reshape(n_samples, n_features).astype(np.float32)
    Y = data['re78'].values.astype(np.float32)
    W = data['re75'].values.reshape(n_samples, n_features).astype(np.float32)

    # 提前标准化和计算带宽（仅一次计算）
    scaler_X = StandardScaler()
    X_scaled = scaler_X.fit_transform(X)
    scaler_W = StandardScaler()
    W_scaled = scaler_W.fit_transform(W)
    sigma_X = np.median(pdist(X_scaled, metric='euclidean'))
    sigma_W = np.median(pdist(W_scaled, metric='euclidean'))
    kernel_X = RBF(length_scale=1.0)
    kernel_W = RBF(length_scale=sigma_W)

    std_Y = np.std(Y)
    std_X = np.std(X)
    t_values = np.linspace(-2 * std_Y, 2 * std_Y, 100, dtype=np.float32)
    s_values = np.linspace(-2 * std_X, 2 * std_X, 100, dtype=np.float32)

    p_value = bootstrap_p_value(X, Y, W, t_values, s_values)
    print(f"p 值: {p_value}")
    if p_value < 0.05:
        print("拒绝零假设")
    else:
        print("未能拒绝零假设")
    

ImportError: cannot import name 'pinv2' from 'scipy.linalg' (c:\Users\lenovo\miniconda\envs\minenv\lib\site-packages\scipy\linalg\__init__.py)

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from multiprocessing import Pool
from tqdm import tqdm  # 导入 tqdm 用于显示进度条
from scipy.spatial.distance import pdist
# 定义全局参数
n_samples = 445  # 样本数量
n_features = 1   # 特征维度
lambda_values = np.logspace(-3, 0, 50)  # 正则化参数的候选值
#lambda_values=np.linspace(4.9e-6, 0.25, 50)
kernel_X = RBF(length_scale=1.0)  # X 的核函数
kernel_W = RBF(length_scale=1.0)  # W 的核函数

# 交叉验证选择最佳 lambda
def cross_validate_lambda(t, X, Y, W, K_X, K_W):
    kf = KFold(n_splits=5)  # 5 折交叉验证
    best_lambda, best_score = None, np.inf
    for lambda_val in lambda_values:
        scores = []
        for train_idx, val_idx in kf.split(X):
            # 分割训练和验证集
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            W_train, W_val = W[train_idx], W[val_idx]
            
            # 计算训练集的核矩阵
            K_X_train = kernel_X(X_train)
            K_W_train = kernel_W(W_train)
            K_W_val_train = kernel_W(W_val, W_train)
            
            # 构造线性系统的矩阵和右端项
            matrix = K_W_train @ K_X_train @ K_W_train + lambda_val * n_samples**2 * K_W_train
            right_hand = K_W_train @ K_X_train @ np.exp(1j * t * Y_train)
            
            # 使用伪逆求解 alpha
            alpha = pinv(matrix) @ right_hand
            
            # 计算验证集的预测误差
            H_W_val = K_W_val_train @ alpha
            Delta_val = np.exp(1j * t * Y_val) - H_W_val
            scores.append(np.mean(np.abs(Delta_val)**2))
        
        # 计算平均得分，选择最佳 lambda
        avg_score = np.mean(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lambda_val
    return best_lambda

# 计算 Delta_phi_m
def compute_Delta_phi_m(X, Y, W, t_values, s_values, K_X, K_W):
    Delta_values = []
    for t in t_values:
        # 获取最佳正则化参数
        best_lambda = cross_validate_lambda(t, X, Y, W, K_X, K_W)
        
        # 构造线性系统的矩阵和右端项
        matrix = K_W @ K_X @ K_W + best_lambda * n_samples**2 * K_W
        right_hand = K_W @ K_X @ np.exp(1j * t * Y)
        
        # 使用伪逆求解 alpha
        alpha = pinv(matrix) @ right_hand
        
        # 计算 U 和 Tn
        U = np.exp(1j * t * Y) - K_W @ alpha
        m_X_s = np.exp(1j * s_values * X[:, None])
        Tn = (1 / np.sqrt(n_samples)) * np.sum(U[:, None] * m_X_s, axis=0)
        
        # 近似积分
        integral_approx = np.sum(np.abs(Tn)**2) * (s_values[1] - s_values[0])
        Delta_values.append(integral_approx)
    return max(Delta_values)

# 自举法中的 Delta 计算（顶层函数）
def compute_bootstrap_Delta(X, Y, W, t_values, s_values, n_samples, kernel_X, kernel_W):
    indices = np.random.choice(n_samples, n_samples, replace=True)
    X_boot, Y_boot, W_boot = X[indices], Y[indices], W[indices]
    K_X_boot, K_W_boot = kernel_X(X_boot), kernel_W(W_boot)
    return compute_Delta_phi_m(X_boot, Y_boot, W_boot, t_values, s_values, K_X_boot, K_W_boot)

# 自举法计算 p 值
def bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps=500):
    # 计算观测值 Delta
    K_X = kernel_X(X)
    K_W = kernel_W(W)
    Delta_obs = compute_Delta_phi_m(X, Y, W, t_values, s_values, K_X, K_W)
    
    # 并行计算自举 Delta，并显示进度条
    with Pool(processes=4) as pool:
        # 使用 tqdm 包装 starmap 的结果
        bootstrap_Deltas = list(tqdm(
            pool.starmap(
                compute_bootstrap_Delta,
                [(X, Y, W, t_values, s_values, n_samples, kernel_X, kernel_W) for _ in range(n_bootstraps)]
            ),
            total=n_bootstraps,
            desc="自举进度"  # 中文进度条描述
        ))
    
    # 计算 p 值
    p_value = np.mean([1 if Delta_boot >= Delta_obs else 0 for Delta_boot in bootstrap_Deltas])
    return p_value

# 主程序
if __name__ == "__main__":
    # 读取数据
    data = pd.read_csv('LaLonde.csv')
    X = data['treat'].values.reshape(n_samples, n_features)  # 处理变量
    Y = data['re78'].values  # 结果变量
    W = data['re75'].values.reshape(n_samples, n_features)  # 协变量
    print(X)
    distances_X = pdist(X, metric='euclidean')  # 计算 X 的成对欧几里得距离
    sigma_X = np.median(distances_X)
    print(sigma_X)

    # 计算 t 和 s 的范围
    std_Y, std_X = np.std(Y), np.std(X)
    t_values = np.linspace(-2*std_Y, 2*std_Y, 5)
    s_values = np.linspace(-2*std_X, 2*std_X, 50)
    
    # 计算 p 值并输出结果
    p_value = bootstrap_p_value(X, Y, W, t_values, s_values, n_bootstraps=500)
    print(f"p 值: {p_value}")
    print("拒绝零假设" if p_value < 0.05 else "未能拒绝零假设")

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.